In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import re
import pymongo

In [2]:
def finding_raw_urls_from_base(url):
    service = Service("./chromedriver/chromedriver.exe")
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(url)
    
    WebDriverWait(driver, 60).until(
        EC.presence_of_all_elements_located((By.TAG_NAME, 'a'))
    )
    
    html = driver.page_source
    driver.quit()
    
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.find_all('a', href=True)
    raw_urls = []
    for link in links:
        if link['href'].startswith("https://www.moneycontrol.com/news/business/"):
            raw_urls.append(link['href'])
    return list(set(raw_urls))
    

In [3]:
raw_urls = finding_raw_urls_from_base("https://www.moneycontrol.com/news/business/stocks/")
# raw_urls = finding_raw_urls_from_base("https://www.moneycontrol.com/news/business/stocks/page-30")

In [4]:
raw_urls

['https://www.moneycontrol.com/news/business/uday-kotak-wants-india-inc-to-go-for-growth-to-fire-up-animal-spirits-in-year-end-musing-12899848.html',
 'https://www.moneycontrol.com/news/business/companies',
 'https://www.moneycontrol.com/news/business/stocks/agri-picks-report-january-02-2025-geojit-financial-services-12901949.html',
 'https://www.moneycontrol.com/news/business/markets/top-25-stocks-to-bet-on-in-2025-12901545.html',
 'https://www.moneycontrol.com/news/business/stocks/railtel-corporation-shares-in-focus-on-winning-order-worth-rs-78-43-crore-12901745.html',
 'https://www.moneycontrol.com/news/business/banks/',
 'https://www.moneycontrol.com/news/business/stocks/page-7/',
 'https://www.moneycontrol.com/news/business/markets/taking-stock-sensex-gains-368-pts-nifty-above-23-700-autos-shine-realty-drags-12901594.html',
 'https://www.moneycontrol.com/news/business/stocks/page-9/',
 'https://www.moneycontrol.com/news/business/markets/here-are-the-top-global-cues-for-today-s-tra

In [5]:
stock_urls =[]
market_urls = []
next_page_urls = []
other_urls = []
ipo_urls = []
def extract_urls(all_urls):
    for link in raw_urls:
        if link.startswith("https://www.moneycontrol.com/news/business/stocks/page"):
            next_page_urls.append(link)
        elif link.startswith("https://www.moneycontrol.com/news/business/stocks/"):
            stock_urls.append(link)
        elif link.startswith("https://www.moneycontrol.com/news/business/markets/"):
            market_urls.append(link)
        elif link.startswith("https://www.moneycontrol.com/news/business/ipo/"):
            ipo_urls.append(link)
        else:
            other_urls.append(link)

In [6]:
extract_urls(raw_urls)

In [7]:
stock_urls

['https://www.moneycontrol.com/news/business/stocks/agri-picks-report-january-02-2025-geojit-financial-services-12901949.html',
 'https://www.moneycontrol.com/news/business/stocks/railtel-corporation-shares-in-focus-on-winning-order-worth-rs-78-43-crore-12901745.html',
 'https://www.moneycontrol.com/news/business/stocks/accumulate-larsen-and-toubro-ltd-target-of-rs-4565-asit-c-mehta-12901524.html',
 'https://www.moneycontrol.com/news/business/stocks/buy-polycab-india-target-of-rs-8300-sharekhan-2-12901531.html',
 'https://www.moneycontrol.com/news/business/stocks/',
 'https://www.moneycontrol.com/news/business/stocks/why-will-water-infra-stocks-be-in-focus-in-2025-12901871.html',
 'https://www.moneycontrol.com/news/business/stocks/buy-metro-brands-target-of-rs-1460-motilal-oswal-2-12901629.html',
 'https://www.moneycontrol.com/news/business/stocks/sky-gold-shining-growth-prospects-12901885.html',
 'https://www.moneycontrol.com/news/business/stocks/buy-apl-apollo-tubes-target-of-rs-1850

In [8]:

regex = r'https:\/\/www\.moneycontrol\.com\/news\/business\/stocks\/[^\/\s]+(?:\.[a-z]{2,6})(?:[\/\?].*)?'

final_stocks_urls = [url for url in stock_urls if re.match(regex, url)]

In [9]:
final_stocks_urls

['https://www.moneycontrol.com/news/business/stocks/agri-picks-report-january-02-2025-geojit-financial-services-12901949.html',
 'https://www.moneycontrol.com/news/business/stocks/railtel-corporation-shares-in-focus-on-winning-order-worth-rs-78-43-crore-12901745.html',
 'https://www.moneycontrol.com/news/business/stocks/accumulate-larsen-and-toubro-ltd-target-of-rs-4565-asit-c-mehta-12901524.html',
 'https://www.moneycontrol.com/news/business/stocks/buy-polycab-india-target-of-rs-8300-sharekhan-2-12901531.html',
 'https://www.moneycontrol.com/news/business/stocks/why-will-water-infra-stocks-be-in-focus-in-2025-12901871.html',
 'https://www.moneycontrol.com/news/business/stocks/buy-metro-brands-target-of-rs-1460-motilal-oswal-2-12901629.html',
 'https://www.moneycontrol.com/news/business/stocks/sky-gold-shining-growth-prospects-12901885.html',
 'https://www.moneycontrol.com/news/business/stocks/buy-apl-apollo-tubes-target-of-rs-1850-sharekhan-3-12901540.html',
 'https://www.moneycontrol

In [10]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["new_db"]
mycol = mydb['raw_news']

In [11]:
service = Service("./chromedriver/chromedriver.exe")
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.199 Safari/537.36")

driver = webdriver.Chrome(service=service, options= options)

In [12]:
def extract_data(url):  
    driver.get(url)
    
    WebDriverWait(driver, 60).until(
        lambda d: d.execute_script("return document.readyState") == "complete"
    )
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')    
    news = {}
    
    title = soup.find('h1', class_ = "article_title")
    news.update({"title":f"{title.text.strip()}"})
    
    desc = soup.find('h2', class_ = 'article_desc')
    news.update({"desc":f"{desc.text.strip()}"})
    
    date_time_div = soup.find('div', class_="article_schedule")
    if date_time_div:
        span_tag = date_time_div.find('span')
        date = span_tag.text.strip()
        news.update({"date":f"{date}"})
    
    datetime = date_time_div.text.strip()
    news.update({"datetime":f"{datetime}"})
    
    paragraphs_list = []
    paragrphs_div = soup.find('div', class_ ="content_wrapper")
    if paragrphs_div:
        paragraph_tags = paragrphs_div.find_all('p')
        for p in paragraph_tags:
            para_text = p.text.strip()            
            if len(para_text) < 50 :
                continue            
            if re.search(r"(click\s+here|disclaimer|modal|window|advertisement|investment\s+tips)", para_text, re.IGNORECASE):
                continue            
            paragraphs_list.append(para_text)         
            
    news.update({"content": paragraphs_list})
            
    # driver.quit()   
    return news

In [13]:
for url in final_stocks_urls :
    data = extract_data(url)
    x = mycol.insert_one(data)
    print(x.inserted_id)

67762b3c1c6d168c109f0dca


67762b431c6d168c109f0dcb


67762b651c6d168c109f0dcc


67762b681c6d168c109f0dcd


67762b6a1c6d168c109f0dce


67762b6d1c6d168c109f0dcf


67762b701c6d168c109f0dd0


67762b751c6d168c109f0dd1


67762b781c6d168c109f0dd2


In [14]:
driver.quit()   